<a href="https://colab.research.google.com/github/kechase/Chase_DSPN_S25/blob/main/ExerciseSubmissions/KChase_Exercise_10_Mixed_effects_mixed_effects_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 10: Mixed effects

1. Loading and formatting data 1/1
2. Model fitting 4/4
3. Model assessment 4/4
4. Reflection 1/1

This homework assignment is designed to give you practice fitting and interpreting mixed effects models.

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again.

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file.

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**.

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [ ]:

# Load the packages
library(tidyverse)
library(dplyr)

# Load the data
lex_data <<- read.csv("/Users/katie/Documents/workspace/Data-Science-for-Psychology-and-Neuro/Homework-Datasets/lex_data/LexicalData.csv")
items <<- read.csv("/Users/katie/Documents/workspace/Data-Science-for-Psychology-and-Neuro/Homework-Datasets/lex_data/Items.csv")

# Check column names
colnames(lex_data)
colnames(items)

# Remove commas from D_RT and convert to numeric
lex_data$D_RT <- as.numeric(gsub(',', '', lex_data$D_RT))

# Select relevant columns from items
items <- items %>%
  select(Word, Length, Log_Freq_HAL)

# Join datasets
word_data <- left_join(lex_data, items, by = c("D_Word" = "Word"))

# Check the result
head(word_data)


[1] "Sub_ID"   "Trial"    "Type"     "D_RT"     "D_Word"   "Outlier"  "D_Zscore"

[1] "Occurrences"  "Word"         "Length"       "Freq_HAL"     "Log_Freq_HAL"

,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore,Length,Log_Freq_HAL
,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>
1,157,1,1,710,browse,false,-0.437,6,8.856
2,67,1,1,1094,refrigerant,false,0.825,11,4.644
3,120,1,1,587,gaining,false,-0.645,7,8.304
4,21,1,1,984,cheerless,false,0.025,9,2.639
5,236,1,1,577,pattered,false,-0.763,8,1.386
6,236,2,1,715,conjures,false,-0.364,8,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output.

In [ ]:
# Fit linear model with interaction term
model <- lm(D_RT ~ Log_Freq_HAL * Length, data = word_data)

# View model summary
summary(model)


Call:
lm(formula = D_RT ~ Log_Freq_HAL * Length, data = word_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Now, install `lme4` using `install.packages()` and then load the library.

In [ ]:
install.packages("nlme")
library(nlme)

# I'm working in VSCode and there is a dependency issue with lme4 that I can't resolve.
# I tried switching to Colab but I can't get the data to load there either.
# I'm not sure what to do about this so, after 2+ hours of troubleshooting,
# I'm using a different, older mixed-effect model library that successfully loads on my computer.

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output.

In [ ]:
# Fit mixed effects model with random intercepts for Subject ID using nlme
mixed_model_alt <- lme(D_RT ~ Length + Log_Freq_HAL, random = ~1|Sub_ID, data = word_data)

# View the model summary
summary(mixed_model_alt)

Linear mixed-effects model fit by REML
  Data: word_data 
       AIC      BIC    logLik
  888475.9 888521.1 -444232.9

Random effects:
 Formula: ~1 | Sub_ID
        (Intercept) Residual
StdDev:    215.2961 288.5925

Fixed effects:  D_RT ~ Length + Log_Freq_HAL 
                Value Std.Error    DF   t-value p-value
(Intercept)  769.0765 13.950437 62309  55.12921       0
Length        29.2257  0.506013 62309  57.75681       0
Log_Freq_HAL -30.1567  0.533005 62309 -56.57870       0
 Correlation: 
             (Intr) Length
Length       -0.379       
Log_Freq_HAL -0.352  0.365

Standardized Within-Group Residuals:
       Min         Q1        Med         Q3        Max 
-4.4812554 -0.5550896 -0.1608760  0.3131659 10.7065695 

Number of Observations: 62610
Number of Groups: 299 

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why?

> *In the linear model, Length had a t-value of 29.175, Log_Freq_HAL: -3.061 and the Interaction (Log_Freq_HAL): -12.528. In the mixed effects model, Length had a t-value of 57.75681, Log_Freq_HAL: -56.57870. In both cases the Length has a positive effect and Log_Freq_HAL a negative effect, but the mixed effects model makes each of those impacts bigger (in their respective directions). The mixed effects model is accounting for fixed & random effects by taking into account within-subjects study designs in a way that the linear model isn't designed to do. *
>

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better?

In [ ]:
ic = AIC(model, mixed_model_alt)
ic
diff(ic$AIC)


Warning message in AIC.default(model, mixed_model_alt):
“models are not all fitted to the same number of observations”


,df,AIC
,<dbl>,<dbl>
model,5,914436.4
mixed_model_alt,5,888475.9


[1] -25960.52

> *The mixed model has a lower AIC and is a better fit model. The mixed-effects model accounts for more variance than the simple linear model, even after accounting for increased complexity.*
>

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set?

> *Maybe you could take into account the change in responses, over time, for the same word or same length of word. I'm running a 100 trial experiment right now and aware that fatigue may be an issue; maybe here, too. *
>

**DUE:** 5pm EST, March 18, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *As always, working with tutor, claude.ai*